# 1. Importing python libaries

In [ ]:
!pip install antropy
!pip install hurst
!pip install statsmodels --upgrade
!pip install scikit-plot
!pip install deap
!pip install scoop

In [2]:
import scipy, csv, statistics, pickle, math, random, numpy, joblib, pywt
from __future__ import unicode_literals
import itertools
from math import factorial
import numpy as np
import pandas as pd
import scipy.signal as ss
import scipy.stats as st
import scikitplot as skplt
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as model_selection
import antropy as ent
import statsmodels.api as sm
from pywt import wavedec
from time import time
from matplotlib import pyplot as plt
from scipy import stats, signal, interpolate
from sklearn import metrics
from scipy.integrate import simps
from sklearn import svm, datasets, metrics, preprocessing
from scipy.stats import entropy, kurtosis
from scipy.signal import butter, sosfilt, sosfreqz, lfilter, find_peaks
from sklearn.decomposition import FastICA
from sklearn.cross_decomposition import CCA
from math import log,e, floor
from hurst import compute_Hc, random_walk
from statsmodels.tsa.ar_model import AutoReg
#from statsmodels.tsa.arima_model import ARIMA
import statsmodels.tsa.arima.model as stats
from scipy.misc import electrocardiogram
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error, accuracy_score, r2_score, confusion_matrix, classification_report, plot_confusion_matrix, f1_score
from sklearn.model_selection import StratifiedKFold, KFold, cross_validate, learning_curve, train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import LogisticRegression
from deap import creator, base, tools, algorithms
from scoop import futures
import warnings
warnings.filterwarnings("ignore")

In [3]:
import imp
imp.find_module('antropy')
# change the hazard coefficient

(None, '/usr/local/lib/python3.7/dist-packages/antropy', ('', '', 5))

In [4]:
# Sample rate and desired cutoff frequencies (in Hz).
fs = 128
lowcut = 0.5
highcut = 45
T = 60
nsamples = T * fs
t = np.linspace(0, T, nsamples, endpoint=False)
a = 0.02
f0 = 128

In [5]:
def butter_bandpass(lowcut, highcut, fs, order = 3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype = 'band', analog = False)
    return b, a
def butter_bandpass_filter(time_series, lowcut, highcut, fs, order = 5):
    b, a = butter_bandpass(lowcut, highcut, fs, order = order)
    y = lfilter(b, a, time_series)
    return y
def eye_movement_artifact(time_data): # parameter must be an 2D array like 32_channels*7860_data
    time_data = time_data.transpose() # Inverse that 2D array
    ica = FastICA()
    comps = ica.fit_transform(time_data)
    data_after = comps.transpose()  # Invert the array 
    return data_after

In [6]:
def signal_pro(data):
    mean_value = 0
    # do the bandpass filter
    for i in range(40):
        for j in range(32):
            data[i][j] = butter_bandpass_filter(data[i][j], lowcut, highcut, fs, order=5)
    # creating dummy variable which contains same data information 
    error_eye =  np.zeros((40,32,7680))
    new_data =  np.zeros((40,32,7680))
    for i in range(40):
        for j in range(32):
            for k in range(7680):
                error_eye[i][j][k] = data[i][j][k]
                new_data[i][j][k] = data[i][j][k]
    for i in range(40):
        error_eye[i] = eye_movement_artifact(error_eye[i])
    for i in range(40):
        for j in range(32):
            mean_value = np.mean(error_eye[i][j])
            for k in range(7680):
                if(data[i][j][k] > 0.0):
                    # data is positive
                    new_data[i][j][k] = data[i][j][k] - abs(mean_value)
                else: # data is negative
                    new_data[i][j][k] = data[i][j][k] + abs(mean_value)
    return new_data

In [7]:
def energy(data): # The energy is the summation of a squared signal
    energy_value = 0
    energy_value = sum(abs(data)**2)
    return energy_value
def average_power(data):  # The average power is the signal mean square
    avg_power = 0
    avg_power = sum(abs(data)**2)
    return (avg_power/data.shape[0])
def first_difference(data):
    temp = 0
    fd = 0
    for i in range(1,data.shape[0] - 1):
        temp = abs(data[i+1]-data[i])
        fd = fd + temp
    return fd/data.shape[0]
def second_difference(data):
    temp, sd = 0, 0
    for i in range(1,data.shape[0] - 2):
        temp = abs(data[i+1]-data[i])
        sd = sd + temp
    return sd/data.shape[0]
def katz_fractal_dimension(data, axis=-1):  # Katz Fractal Dimension
    x = np.asarray(data)
    dists = np.abs(np.diff(x, axis=axis))
    ll = dists.sum(axis=axis)
    ln = np.log10(ll / dists.mean(axis=axis))
    aux_d = x - np.take(x, indices=[0], axis=axis)
    d = np.max(np.abs(aux_d), axis=axis)
    kfd = np.squeeze(ln / (ln + np.log10(d / ll)))
    if not kfd.ndim:
        kfd = kfd.item()
    return kfd
def non_linear_energy(data): # Nonlinear Energy
    nle = 0
    nle_value = 0 
    for i in range(1,data.shape[0]-1):
        nle = (data[i]**2)-(data[i+1]*data[i-1])
        nle_value = nle + nle_value
    return nle_value

In [8]:
def shannon_entopy(data):     # Shannon Entropy
    pd_series = pd.Series(data)
    counts = pd_series.value_counts()
    return entropy(counts)    # scipy.stats.entropy(counts) is called
def _embed(x, order = 3, delay = 1):
    N = len(x)
    Y = np.empty((order, N - (order - 1) * delay))
    for i in range(order):
        Y[i] = x[i * delay:i * delay + Y.shape[1]]
    return Y.T
def util_pattern_space(time_series, lag, dim):
    n = len(time_series)
    if lag * dim > n:
        raise Exception('Result matrix exceeded size limit, try to change lag or dim.')
    elif lag < 1:
        raise Exception('Lag should be greater or equal to 1.')
    pattern_space = np.empty((n - lag * (dim - 1), dim))
    for i in range(n - lag * (dim - 1)):
        for j in range(dim):
            pattern_space[i][j] = time_series[i + j * lag]
    return pattern_space
def util_standardize_signal(time_series):
    return (time_series - np.mean(time_series)) / np.std(time_series)
def util_granulate_time_series(time_series, scale):
    n = len(time_series)
    b = int(np.fix(n / scale))
    temp = np.reshape(time_series[0:b*scale], (b, scale))
    cts = np.mean(temp, axis = 1)
    return cts
def util_rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
def shannon_entropy_WPE(time_series):
    if not isinstance(time_series, str): # Check if string
        time_series = list(time_series)
    data_set = list(set(time_series))  # Create a frequency data
    freq_list = []
    for entry in data_set:
        counter = 0.
        for i in time_series:
            if i == entry:
                counter += 1
        freq_list.append(float(counter) / len(time_series))
    ent = 0.0 # Shannon entropy
    for freq in freq_list:
        ent += freq * np.log2(freq)
    ent = -ent
    return ent

In [9]:
def sample_entropy_WPE(time_series, sample_length = 7680, tolerance = None):
    #The code below follows the sample length convention of Ref [1] so:
    M = sample_length - 1;
    time_series = np.array(time_series)
    if tolerance is None:
        tolerance = 0.1*np.std(time_series)
    n = len(time_series)
    #Ntemp is a vector that holds the number of matches. N[k] holds matches templates of length k
    Ntemp = np.zeros(M + 2)
    #Templates of length 0 matches by definition:
    Ntemp[0] = n*(n - 1) / 2
    for i in range(n - M - 1):
        template = time_series[i:(i+M+1)];#We have 'M+1' elements in the template
        rem_time_series = time_series[i+1:]
        searchlist = np.nonzero(np.abs(rem_time_series - template[0]) < tolerance)[0]
        go = len(searchlist) > 0;
        length = 1;
        Ntemp[length] += len(searchlist)
        while go:
            length += 1
            nextindxlist = searchlist + 1;
            nextindxlist = nextindxlist[nextindxlist < n - 1 - i]#Remove candidates too close to the end
            nextcandidates = rem_time_series[nextindxlist]
            hitlist = np.abs(nextcandidates - template[length-1]) < tolerance
            searchlist = nextindxlist[hitlist]
            Ntemp[length] += np.sum(hitlist)
            go = any(hitlist) and length < M + 1
    sampen =  - np.log(Ntemp[1:] / Ntemp[:-1])
    return sampen

In [10]:
def multiscale_entropy(time_series, sample_length =7680, tolerance = None, maxscale = None):
    if tolerance is None:
        #we need to fix the tolerance at this level. If it remains 'None' it will be changed in call to sample_entropy()
        tolerance = 0.1*np.std(time_series)
    if maxscale is None:
        maxscale = len(time_series)
    mse = np.zeros(maxscale)
    for i in range(maxscale):
        temp = util_granulate_time_series(time_series, i+1)
        mse[i] = sample_entropy_WPE(temp, sample_length, tolerance)[-1]
    return mse
def permutation_entropy_WPE(time_series, order = 3, delay = 1, normalize = False):
    x = np.array(time_series)
    hashmult = np.power(order, np.arange(order))
    # Embed x and sort the order of permutations
    sorted_idx = _embed(x, order=order, delay=delay).argsort(kind='quicksort')
    # Associate unique integer to each permutations
    hashval = (np.multiply(sorted_idx, hashmult)).sum(1)
    # Return the counts
    _, c = np.unique(hashval, return_counts=True)
    # Use np.true_divide for Python 2 compatibility
    p = np.true_divide(c, c.sum())
    pe = -np.multiply(p, np.log2(p)).sum()
    if normalize:
        pe /= np.log2(factorial(order))
    return pe

In [11]:
def weighted_permutation_entropy(time_series, order=3, normalize=False):
    x = np.array(time_series)
    hashmult = np.power(order, np.arange(order))
    embedded = _embed(x, order=order)
    sorted_idx = embedded.argsort(kind='quicksort')
    weights = np.var(util_rolling_window(x, order), 1)
    hashval = (np.multiply(sorted_idx, hashmult)).sum(1)
    mapping = {}
    for i in np.unique(hashval):
        mapping[i] = np.where(hashval == i)[0] 
    weighted_counts = dict.fromkeys(mapping)
    for k, v in mapping.items():
        weighted_count = 0
        for i in v:
            weighted_count += weights[i]
        weighted_counts[k] = weighted_count
    weighted_counts_array = np.array(list(weighted_counts.values()))
    p = np.true_divide(weighted_counts_array, weighted_counts_array.sum())
    pe = -np.multiply(p, np.log2(p)).sum()
    if normalize:
        pe /= np.log2(factorial(order))
    return pe

In [12]:
def hjorth_params(x, axis=-1):
    x = np.asarray(x)
    # Calculate derivatives
    dx = np.diff(x, axis=axis)
    ddx = np.diff(dx, axis=axis)
    # Calculate variance
    x_var = np.var(x, axis=axis)  # = activity
    dx_var = np.var(dx, axis=axis)
    ddx_var = np.var(ddx, axis=axis)
    # Mobility and complexity
    act = x_var
    mob = np.sqrt(dx_var / x_var)
    com = np.sqrt(ddx_var / dx_var) / mob
    return act, mob, com

In [13]:
def petrosian_fd(data, axis=-1):  # Petrosian fractal dimension
    x = np.asarray(data)
    N = x.shape[axis]
    nzc_deriv = ent.num_zerocross(np.diff(x, axis=axis), axis=axis) # Number of sign changes in the first derivative of the signal
    pfd = np.log10(N) / (np.log10(N) + np.log10(N / (N + 0.4 * nzc_deriv)))
    return pfd
def hjorth_parameter(data): # Hjorth mobility and complexity
    hjorth_value = ent.hjorth_params(data)
    return (hjorth_value[0],hjorth_value[1])
def extrema(arr): # function to find local extremum
    n = arr.shape[0]
    count = 0
    a = arr.tolist()
    for i in range(1, n - 1) :  # start loop from position 1 till n-1
        # only one condition will be true at a time either a[i] will be greater than neighbours or less than neighbours
        # check if a[i] if greater than both its neighbours, then add 1 to x
        count += (a[i]>a[i-1] and a[i]>a[i+1]);
        # check if a[i] if less than both its neighbours, then add 1 to x
        count += (a[i] < a[i - 1] and a[i] < a[i + 1]);
    return count

In [14]:
def energy_in_each_frq_band(psd,frqs):
    # PSD  -->   Frequency Bands  -->  energy
    delta_low_freq, delta_high_freq = 0.5, 4
    theta_low_freq, theta_high_freq = 4, 8
    alpha_low_freq, alpha_high_freq = 8, 12
    beta_low_freq , beta_high_freq  = 12, 30
    gamma_low_freq , gamma_high_freq = 30, 48
    idx_delta = np.logical_and(freqs >= delta_low_freq, freqs <= delta_high_freq)
    idx_theta = np.logical_and(freqs >= theta_low_freq, freqs <= theta_high_freq)
    idx_alpha = np.logical_and(freqs >= alpha_low_freq, freqs <= alpha_high_freq)
    idx_beta  = np.logical_and(freqs >= beta_low_freq, freqs <= beta_high_freq)
    idx_gamma = np.logical_and(freqs >= gamma_low_freq, freqs <= gamma_high_freq)
    delta_energy = energy(psd[idx_delta])
    theta_energy = energy(psd[idx_theta])
    alpha_energy = energy(psd[idx_alpha])
    beta_energy = energy(psd[idx_beta])
    gamma_energy = energy(psd[idx_gamma])
    return delta_energy, theta_energy, alpha_energy, beta_energy, gamma_energy
def IWMF(psd,frqs):
    iwmf, temp = 0, 0
    for i in range(psd.shape[0]):
        temp = psd[i]*frqs[i]
        iwmf = iwmf + temp
    return iwmf
def IWBW(psd, frqs):
    iwbw_1 = 0
    iwmf = IWMF(psd, frqs)
    for i in range(psd.shape[0]):
        temp_1 = (frqs[i]-iwmf)**2
        temp_2 = temp_1*psd[i]
        iwbw_1 = temp_2 + iwbw_1
    return math.sqrt(iwbw_1)

In [15]:
def calcNormalizedFFT(epoch, lvl, nt, fs=128):
    lseg = np.round(nt/fs*lvl).astype('int')
    D = np.absolute(np.fft.fft(epoch, n=lseg[-1]))
    D /= D.sum()
    return D
def SpectralEdgeFreq(epoch, lvl): # find the spectral edge frequency
    nt, fs, percent, tfreq = 18, 512, 0.5, 40
    sfreq, ppow= fs, percent
    topfreq = int(round(nt/sfreq*tfreq)) + 1
    D = calcNormalizedFFT(epoch, lvl, nt, fs)
    A = np.cumsum(D[:topfreq])
    B = A - (A.max()*ppow)
    spedge = np.min(np.abs(B))
    spedge = (spedge - 1)/(topfreq - 1)*tfreq
    return spedge
def DWT(x):
    resp = pywt.dwt(x, 'db4') # single level decompositions
    resp = np.ravel(np.array(resp[0])) 
    shyam = pywt.wavedec(resp, wavelet = 'db4', level=3) # it will return 4 labels of decompositions
    return (shyam[0], shyam[1], shyam[2], shyam[3])

In [16]:
def sf_psd(psd):
    mean_psd = np.mean(psd)
    var_psd = np.var(psd)
    mode_psd = float(st.mode(psd)[0])
    median_psd = np.median(psd)
    skew_psd = scipy.stats.skew(psd)
    std_psd = np.std(psd)
    kurtosis_psd = kurtosis(psd)
    f_d_psd = first_difference(psd) 
    nfd_psd = f_d_psd/std_psd
    s_d_psd = second_difference(psd)
    nsd_psd = s_d_psd/std_psd
    return mean_psd, var_psd, mode_psd, median_psd, skew_psd, std_psd, kurtosis_psd, f_d_psd, nfd_psd, s_d_psd, nsd_psd

In [17]:
def sf_dwt(dwt):
    mean_dwt = np.mean(dwt)
    var_dwt = np.var(dwt)
    mode_dwt = float(st.mode(dwt)[0])
    median_dwt = np.median(dwt)
    skew_dwt = scipy.stats.skew(dwt)
    std_dwt = np.std(dwt)
    kurtosis_dwt = kurtosis(dwt)
    f_d_dwt = first_difference(dwt)
    nfd_dwt = f_d_dwt/std_dwt
    s_d_dwt = second_difference(dwt)
    nsd_dwt = s_d_dwt/std_dwt
    return mean_dwt, var_dwt, mode_dwt, median_dwt, skew_dwt, std_dwt, kurtosis_dwt, f_d_dwt, nfd_dwt, s_d_dwt, nsd_dwt

In [18]:
def get_features(data, data_dwt, channel_no):
  feature_vector = []
  no_of_features = 150
  feature = np.ones((40, len(channel_no)*no_of_features))
  delta_left, theta_left, alpha_left, beta_left, gamma_left, delta_right, theta_right, alpha_right, beta_right, gamma_right = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
  for video in range(0, 40):
    channel_count = 0
    for channel in channel_no:
      coeffs = wavedec(data_dwt[video, channel], 'db1', level = 4)
      delta, theta, alpha, beta, gamma = coeffs
      #-----------------------------------------------------------------------------------------------------------------------
      # 1. Statistical feature are computed from the DWT feature which is decomposed over 0-64 Hz data
      delta_mean_dwt, delta_var_dwt, delta_mode_dwt, delta_median_dwt, delta_skew_dwt, delta_std_dwt, delta_kurtosis_dwt, delta_f_d_dwt, delta_nfd_dwt, delta_s_d_dwt, delta_nsd_dwt = sf_dwt(delta)
      theta_mean_dwt, theta_var_dwt, theta_mode_dwt, theta_median_dwt, theta_skew_dwt, theta_std_dwt, theta_kurtosis_dwt, theta_f_d_dwt, theta_nfd_dwt, theta_s_d_dwt, theta_nsd_dwt = sf_dwt(theta)
      alpha_mean_dwt, alpha_var_dwt, alpha_mode_dwt, alpha_median_dwt, alpha_skew_dwt, alpha_std_dwt, alpha_kurtosis_dwt, alpha_f_d_dwt, alpha_nfd_dwt, alpha_s_d_dwt, alpha_nsd_dwt = sf_dwt(alpha) 
      beta_mean_dwt , beta_var_dwt,  beta_mode_dwt,  beta_median_dwt,  beta_skew_dwt,  beta_std_dwt,  beta_kurtosis_dwt,  beta_f_d_dwt,  beta_nfd_dwt,  beta_s_d_dwt,  beta_nsd_dwt  = sf_dwt(beta)
      gamma_mean_dwt, gamma_var_dwt, gamma_mode_dwt, gamma_median_dwt, gamma_skew_dwt, gamma_std_dwt, gamma_kurtosis_dwt, gamma_f_d_dwt, gamma_nfd_dwt, gamma_s_d_dwt, gamma_nsd_dwt = sf_dwt(gamma)
      #----------------------------------------------------------------------------------------------------------------------
      
      # 2. Energy, Average Power  and RMS
      # 2.1. Energy calculation for each band
      delta_dwt_energy, theta_dwt_energy, alpha_dwt_energy, beta_dwt_energy, gamma_dwt_energy = energy(delta), energy(theta), energy(alpha), energy(beta), energy(gamma)

      # 2.2. Average power for each band
      delta_dwt_avg_power, theta_dwt_avg_power, alpha_dwt_avg_power, beta_dwt_avg_power, gamma_dwt_avg_power = average_power(delta), average_power(theta), average_power(alpha), average_power(beta), average_power(gamma)

      # 2.3. RMS value for each band
      delta_dwt_rms, theta_dwt_rms, alpha_dwt_rms, beta_dwt_rms, gamma_dwt_rms = np.sqrt(np.mean(delta**2)), np.sqrt(np.mean(theta**2)), np.sqrt(np.mean(alpha**2)), np.sqrt(np.mean(beta**2)), np.sqrt(np.mean(gamma**2))

      # 3. shEn, ApEn , PE, WPE
      # 3.1 Shannon entropy (shEn)
      delta_dwt_ShEn, theta_dwt_ShEn, alpha_dwt_ShEn, beta_dwt_ShEn, gamma_dwt_ShEn = shannon_entopy(delta), shannon_entopy(theta), shannon_entopy(alpha), shannon_entopy(beta), shannon_entopy(gamma)

      # 3.2 Approximate entropy
      delta_dwt_aentropy, theta_dwt_aentropy, alpha_dwt_aentropy, beta_dwt_aentropy, gamma_dwt_aentropy = ent.app_entropy(delta), ent.app_entropy(theta), ent.app_entropy(alpha), ent.app_entropy(beta), ent.app_entropy(gamma)

      # 3.3 Permutation entropy
      delta_dwt_pentropy, theta_dwt_pentropy, alpha_dwt_pentropy, beta_dwt_pentropy, gamma_dwt_pentropy = ent.perm_entropy(delta, normalize = True), ent.perm_entropy(theta, normalize=True), ent.perm_entropy(alpha, normalize=True), ent.perm_entropy(beta, normalize=True), ent.perm_entropy(gamma, normalize=True)

      # 3.4 Weigheted Permutation Entropy 
      delta_dwt_wpe, theta_dwt_wpe, alpha_dwt_wpe = weighted_permutation_entropy(delta, order = 3, normalize = False), weighted_permutation_entropy(theta, order = 3, normalize=False), weighted_permutation_entropy(alpha, order=3, normalize=False)
      beta_dwt_wpe, gamma_dwt_wpe = weighted_permutation_entropy(beta, order = 3, normalize = False), weighted_permutation_entropy(gamma, order = 3, normalize = False)

      # 4. Hurst Exponent(HE)
      # Here we have two paramaters of HE i.e. H and c
      H_delta, c_delta, data_HC_delta = compute_Hc(delta, kind = 'change', simplified = True)
      H_theta, c_theta, data_HC_theta = compute_Hc(theta, kind = 'change', simplified = True)
      H_alpha, c_alpha, data_HC_alpha = compute_Hc(alpha, kind = 'change', simplified = True)
      H_beta, c_beta, data_HC_beta = compute_Hc(beta, kind = 'change', simplified = True)
      H_gamma, c_gamma, data_HC_gamma = compute_Hc(gamma, kind = 'change', simplified = True)
        
      # 5. Fractal dimention
      higuchi_delta   = ent.higuchi_fd(delta)   # Higuchi fractal dimension for delta
      petrosian_delta = petrosian_fd(delta)   # Petrosian fractal dimension for delta
      higuchi_theta   = ent.higuchi_fd(theta)   # Higuchi fractal dimension for theta
      petrosian_theta = petrosian_fd(theta)   # Petrosian fractal dimension for theta
      higuchi_alpha   = ent.higuchi_fd(alpha)   # Higuchi fractal dimension for alpha
      petrosian_alpha = petrosian_fd(alpha)   # Petrosian fractal dimension for alpha
      higuchi_beta   = ent.higuchi_fd(beta)   # Higuchi fractal dimension for beta
      petrosian_beta = petrosian_fd(beta)   # Petrosian fractal dimension for beta
      higuchi_gamma   = ent.higuchi_fd(gamma)   # Higuchi fractal dimension for gamma
      petrosian_gamma = petrosian_fd(gamma)   # Petrosian fractal dimension for gamma
        
      # 6. Auto regressive (AR)
      res_delta = AutoReg(delta,lags = 128).fit()
      res_theta = AutoReg(theta,lags = 128).fit()
      res_alpha = AutoReg(alpha,lags = 128).fit()
      res_beta = AutoReg(beta,lags = 128).fit()
      res_gamma = AutoReg(gamma,lags = 128).fit()
      aic_delta_ar, hqic_delta_ar, bic_delta_ar, llf_delta_ar = res_delta.aic, res_delta.hqic, res_delta.bic, res_delta.llf
      aic_theta_ar, hqic_theta_ar, bic_theta_ar, llf_theta_ar = res_theta.aic, res_theta.hqic, res_theta.bic, res_theta.llf
      aic_alpha_ar, hqic_alpha_ar, bic_alpha_ar, llf_alpha_ar = res_alpha.aic, res_alpha.hqic, res_alpha.bic, res_alpha.llf
      aic_beta_ar, hqic_beta_ar, bic_beta_ar, llf_beta_ar = res_beta.aic, res_beta.hqic, res_beta.bic, res_beta.llf
      aic_gamma_ar, hqic_gamma_ar, bic_gamma_ar, llf_gamma_ar = res_gamma.aic, res_gamma.hqic, res_gamma.bic, res_gamma.llf


      # 7. Autoregressive moving Average (ARMA)
      try:
        arma_delta = stats.ARIMA(delta, order=(5,1,0)).fit()
      except:
        arma_delta = stats.ARIMA(delta, order=(3,1,0)).fit()
      try:
        arma_theta = stats.ARIMA(theta, order=(5,1,0)).fit()
      except:
        arma_theta = stats.ARIMA(theta, order=(3,1,0)).fit()
      try:
        arma_alpha = stats.ARIMA(alpha, order=(5,1,0)).fit()
      except:
        arma_alpha = stats.ARIMA(alpha, order=(3,1,0)).fit()
      try:
        arma_beta = stats.ARIMA(beta, order=(5,1,0)).fit()
      except:
        arma_beta = stats.ARIMA(beta, order=(3,1,0)).fit()
      try:
        arma_gamma = stats.ARIMA(gamma, order=(5,1,0)).fit()
      except:
        arma_gamma = stats.ARIMA(gamma, order=(3,1,0)).fit()

      aic_delta_arma, hqic_delta_arma, bic_delta_arma, llf_delta_arma = arma_delta.aic, arma_delta.hqic, arma_delta.bic, arma_delta.llf
      aic_theta_arma, hqic_theta_arma, bic_theta_arma, llf_theta_arma = arma_theta.aic, arma_theta.hqic, arma_theta.bic, arma_theta.llf
      aic_alpha_arma, hqic_alpha_arma, bic_alpha_arma, llf_alpha_arma = arma_alpha.aic, arma_alpha.hqic, arma_alpha.bic, arma_alpha.llf
      aic_beta_arma, hqic_beta_arma, bic_beta_arma, llf_beta_arma = arma_beta.aic, arma_beta.hqic, arma_beta.bic, arma_beta.llf
      aic_gamma_arma, hqic_gamma_arma, bic_gamma_arma, llf_gamma_arma = arma_gamma.aic, arma_gamma.hqic, arma_gamma.bic, arma_gamma.llf
      #--------------------------------------------------------------------------------------------------------------------------
      feature_vector = [delta_mean_dwt, delta_var_dwt, delta_mode_dwt, delta_median_dwt, delta_skew_dwt, delta_std_dwt, 
                        delta_kurtosis_dwt, delta_f_d_dwt, delta_nfd_dwt, delta_s_d_dwt, delta_nsd_dwt, theta_mean_dwt, theta_var_dwt, theta_mode_dwt, 
                        theta_median_dwt, theta_skew_dwt, theta_std_dwt, theta_kurtosis_dwt, theta_f_d_dwt, theta_nfd_dwt, theta_s_d_dwt, theta_nsd_dwt,
                        alpha_mean_dwt, alpha_var_dwt, alpha_mode_dwt, alpha_median_dwt, alpha_skew_dwt, alpha_std_dwt, alpha_kurtosis_dwt,
                        alpha_f_d_dwt, alpha_nfd_dwt, alpha_s_d_dwt, alpha_nsd_dwt, beta_mean_dwt, beta_var_dwt, beta_mode_dwt,  
                        beta_median_dwt, beta_skew_dwt, beta_std_dwt, beta_kurtosis_dwt, beta_f_d_dwt, beta_nfd_dwt,  beta_s_d_dwt,
                        beta_nsd_dwt, gamma_mean_dwt, gamma_var_dwt, gamma_mode_dwt, gamma_median_dwt, gamma_skew_dwt, gamma_std_dwt, 
                        gamma_kurtosis_dwt, gamma_f_d_dwt, gamma_nfd_dwt, gamma_s_d_dwt, gamma_nsd_dwt, delta_dwt_energy, theta_dwt_energy,
                        alpha_dwt_energy, beta_dwt_energy, gamma_dwt_energy, delta_dwt_avg_power, theta_dwt_avg_power, alpha_dwt_avg_power, 
                        beta_dwt_avg_power, gamma_dwt_avg_power, delta_dwt_rms, theta_dwt_rms, alpha_dwt_rms, beta_dwt_rms, gamma_dwt_rms,
                        delta_dwt_ShEn, theta_dwt_ShEn, alpha_dwt_ShEn, beta_dwt_ShEn, gamma_dwt_ShEn, delta_dwt_aentropy, theta_dwt_aentropy,
                        alpha_dwt_aentropy, beta_dwt_aentropy, gamma_dwt_aentropy, delta_dwt_pentropy, theta_dwt_pentropy, alpha_dwt_pentropy,
                        beta_dwt_pentropy, gamma_dwt_pentropy, delta_dwt_wpe, theta_dwt_wpe, alpha_dwt_wpe, beta_dwt_wpe, gamma_dwt_wpe,
                        H_delta, c_delta, H_theta, c_theta, H_alpha, c_alpha, H_beta, c_beta, H_gamma, c_gamma, higuchi_delta, petrosian_delta, 
                        higuchi_theta, petrosian_theta, higuchi_alpha, petrosian_alpha, higuchi_beta, petrosian_beta, higuchi_gamma, petrosian_gamma,
                        aic_delta_ar, hqic_delta_ar, bic_delta_ar, llf_delta_ar, aic_theta_ar, hqic_theta_ar, bic_theta_ar, llf_theta_ar, aic_alpha_ar, 
                        hqic_alpha_ar, bic_alpha_ar, llf_alpha_ar, aic_beta_ar, hqic_beta_ar, bic_beta_ar, llf_beta_ar, aic_gamma_ar, hqic_gamma_ar, 
                        bic_gamma_ar, llf_gamma_ar, aic_delta_arma, hqic_delta_arma, bic_delta_arma, llf_delta_arma, aic_theta_arma, hqic_theta_arma, 
                        bic_theta_arma, llf_theta_arma, aic_alpha_arma, hqic_alpha_arma, bic_alpha_arma, llf_alpha_arma, aic_beta_arma, hqic_beta_arma, 
                        bic_beta_arma, llf_beta_arma, aic_gamma_arma, hqic_gamma_arma, bic_gamma_arma, llf_gamma_arma]
      for i in range(0, no_of_features):
        feature[video][channel_count + i] = feature_vector[i]
      channel_count = channel_count + no_of_features
    print("video: ", video)
  return feature

In [20]:
def all_channels_name():
  channels = ['delta_mean_dwt', 'delta_var_dwt', 'delta_mode_dwt', 'delta_median_dwt', 'delta_skew_dwt', 'delta_std_dwt', 
              'delta_kurtosis_dwt', 'delta_f_d_dwt', 'delta_nfd_dwt', 'delta_s_d_dwt', 'delta_nsd_dwt', 'theta_mean_dwt', 'theta_var_dwt', 'theta_mode_dwt', 
              'theta_median_dwt', 'theta_skew_dwt', 'theta_std_dwt', 'theta_kurtosis_dwt', 'theta_f_d_dwt', 'theta_nfd_dwt', 'theta_s_d_dwt', 'theta_nsd_dwt',
              'alpha_mean_dwt', 'alpha_var_dwt', 'alpha_mode_dwt', 'alpha_median_dwt', 'alpha_skew_dwt', 'alpha_std_dwt', 'alpha_kurtosis_dwt',
              'alpha_f_d_dwt', 'alpha_nfd_dwt', 'alpha_s_d_dwt', 'alpha_nsd_dwt', 'beta_mean_dwt', 'beta_var_dwt', 'beta_mode_dwt',  
              'beta_median_dwt', 'beta_skew_dwt', 'beta_std_dwt', 'beta_kurtosis_dwt', 'beta_f_d_dwt', 'beta_nfd_dwt',  'beta_s_d_dwt',
              'beta_nsd_dwt', 'gamma_mean_dwt', 'gamma_var_dwt', 'gamma_mode_dwt', 'gamma_median_dwt', 'gamma_skew_dwt', 'gamma_std_dwt', 
              'gamma_kurtosis_dwt', 'gamma_f_d_dwt', 'gamma_nfd_dwt', 'gamma_s_d_dwt', 'gamma_nsd_dwt',
              'delta_dwt_energy', 'theta_dwt_energy', 'alpha_dwt_energy', 'beta_dwt_energy', 'gamma_dwt_energy', 
              'delta_dwt_avg_power', 'theta_dwt_avg_power', 'alpha_dwt_avg_power', 'beta_dwt_avg_power', 'gamma_dwt_avg_power', 
              'delta_dwt_rms', 'theta_dwt_rms', 'alpha_dwt_rms', 'beta_dwt_rms', 'gamma_dwt_rms',
              'delta_dwt_ShEn', 'theta_dwt_ShEn', 'alpha_dwt_ShEn', 'beta_dwt_ShEn', 'gamma_dwt_ShEn', 
              'delta_dwt_aentropy', 'theta_dwt_aentropy', 'alpha_dwt_aentropy', 'beta_dwt_aentropy', 'gamma_dwt_aentropy', 
              'delta_dwt_pentropy', 'theta_dwt_pentropy', 'alpha_dwt_pentropy', 'beta_dwt_pentropy', 'gamma_dwt_pentropy', 
              'delta_dwt_wpe', 'theta_dwt_wpe', 'alpha_dwt_wpe', 'beta_dwt_wpe', 'gamma_dwt_wpe',
              'H_delta', 'c_delta', 'H_theta', 'c_theta', 'H_alpha', 'c_alpha', 'H_beta', 'c_beta', 'H_gamma', 'c_gamma', 
              'higuchi_delta', 'petrosian_delta', 'higuchi_theta', 'petrosian_theta', 'higuchi_alpha', 'petrosian_alpha', 'higuchi_beta', 'petrosian_beta', 'higuchi_gamma', 'petrosian_gamma',
              'aic_delta_ar', 'hqic_delta_ar', 'bic_delta_ar', 'llf_delta_ar',  'aic_theta_ar', 'hqic_theta_ar', 'bic_theta_ar', 'llf_theta_ar', 'aic_alpha_ar', 
              'hqic_alpha_ar', 'bic_alpha_ar', 'llf_alpha_ar', 'aic_beta_ar', 'hqic_beta_ar', 'bic_beta_ar', 'llf_beta_ar', 'aic_gamma_ar', 'hqic_gamma_ar', 'bic_gamma_ar', 'llf_gamma_ar',
              'aic_delta_arma', 'hqic_delta_arma', 'bic_delta_arma', 'llf_delta_arma', 'aic_theta_arma', 'hqic_theta_arma', 'bic_theta_arma', 'llf_theta_arma',
              'aic_alpha_arma', 'hqic_alpha_arma', 'bic_alpha_arma', 'llf_alpha_arma', 'aic_beta_arma', 'hqic_beta_arma', 'bic_beta_arma', 'llf_beta_arma',
              'aic_gamma_arma', 'hqic_gamma_arma', 'bic_gamma_arma', 'llf_gamma_arma']
  print(len(channels))
  return channels

In [21]:
def get_channels_name(channel_no):
  channels = all_channels_name()
  channels_name = []
  for i in eeg_channels[channel_no]:
    for j in channels:
      channels_name.append(i + "_" + j)
  return channels_name

In [22]:
eeg_channels = np.array(["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7",
                         "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz",
                         "C4", "T8", "CP6", "CP2", "P4", "P8", "PO4", "O2"])
subject_names = ["s01", "s02", "s03", "s04", "s05", "s06", "s07", "s08", "s09", "s10", "s11", "s12",
                 "s13", "s14", "s15", "s16", "s17", "s18", "s19", "s20", "s21",
                "s22", "s23", "s24", "s25", "s26", "s27", "s28", "s29", "s30", "s31", "s32"]
def get_channel_no(channel_name):
  channel_no = []
  for i in range(0, len(eeg_channels)):
    for j in range(0, len(channel_name)):
      if(eeg_channels[i] == channel_name[j]):
        channel_no.append(i)
  return channel_no # is in sorted order

In [23]:
subject_arousal_channels = [['P7', 'CP1', 'P3', 'Fz', 'P8', 'FC2', 'F7', 'C4'], ['FC2'],
                            ['AF4', 'FC1', 'O2', 'F3', 'AF3', 'P8', 'T7', 'C4', 'Pz', 'FC5', 'PO3'], ['P8', 'CP2', 'P7', 'Pz', 'P3'], ['Fz', 'P7', 'T7', 'CP6', 'O2', 'Fp1'], 
                            ['C3', 'AF4', 'AF3', 'P7', 'F3', 'F4', 'CP5', 'T8', 'Oz', 'FC6', 'F7', 'Cz'], ['FC1', 'F3', 'O1', 'FC5', 'P3', 'Fz'], ['T8', 'Fp1', 'FC6', 'Fp2', 'Fz', 'CP2', 'PO3', 'CP1', 'Cz'], ['AF4'],
                            ['FC1', 'Fp1', 'CP5', 'P7', 'T8', 'PO4', 'C3', 'PO3', 'F4', 'Cz'], ['T7'], ['FC6', 'CP5', 'F3', 'Cz', 'FC2'], ['PO3', 'P7', 'Fp2'], ['PO3', 'FC2', 'P3', 'AF3', 'CP5', 'Pz', 'F7', 'T7', 'O2', 'F3'],
                            ['FC2', 'FC1', 'Fz', 'P3', 'AF4', 'Cz', 'CP1', 'CP2', 'Fp2', 'AF3', 'CP5', 'Pz'], ['O2', 'FC1', 'C3', 'C4', 'Cz'], ['P4', 'CP2', 'FC2', 'Fp1'], ['AF4', 'FC6', 'FC5', 'PO4', 'P3', 'CP1'], 
                            ['CP5', 'P8', 'C4', 'FC6', 'PO4', 'O2', 'P4', 'CP6', 'P7', 'CP2', 'PO3', 'FC2'], ['F3', 'P7', 'CP6', 'F4', 'FC6', 'F8', 'CP5', 'Fz', 'T7',  'FC5', 'AF3', 'O1', 'CP1', 'Cz', 'F7', 'C3', 'FC1'], 
                            ['P4', 'Fz'], ['O1', 'CP2', 'FC1', 'CP1', 'CP6', 'F8', 'P8'], ['Fp1', 'F3', 'CP5', 'Cz', 'FC5'],
                            ['Pz', 'O2', 'P8', 'CP2', 'O1', 'PO3', 'F8', 'FC1', 'P3', 'CP1', 'Cz', 'T8', 'FC2', 'C3', 'FC5', 'Fz', 'CP5', 'F4', 'PO4', 'F7', 'Oz', 'T7', 'P7', 'AF4', 'Fp2', 'FC6', 'F3', 'P4', 'Fp1', 'AF3', 'C4', 'CP6'],
                            ['FC2'], ['AF4', 'FC5', 'CP5', 'Oz', 'FC2', 'C3', 'Cz'], ['PO3', 'Oz', 'AF4', 'CP6', 'F8', 'Cz', 'FC6', 'O1', 'P8', 'F4', 'Fp2'],
                            ['PO3', 'F4', 'FC6', 'Oz', 'CP2', 'O2', 'P4', 'CP6'], ['O2', 'CP6', 'Cz', 'PO4', 'O1', 'FC1', 'T8', 'P4', 'F4', 'C3'],
                            ['Oz', 'P4', 'CP1', 'P3', 'FC6', 'AF3', 'F3', 'F7', 'CP2', 'Cz', 'CP5', 'C4', 'FC1', 'P8', 'T8', 'T7', 'F8', 'C3', 'O2', 'CP6', 'Fp1', 'AF4', 'O1', 'F4'],
                            ['FC2', 'C4', 'PO4', 'P4', 'F4', 'FC1', 'C3', 'CP6', 'AF3', 'F7', 'FC5'], ['Fz', 'Cz', 'P7', 'F4', 'F8', 'C4', 'P4']]

In [24]:
# data_dwt is from 0.5hz to 60 Hz
# Sample rate and desired cutoff frequencies (in Hz).
fs_dwt = 128
lowcut_dwt = 0.5
highcut_dwt = 45
T_dwt = 60
nsamples_dwt = T_dwt * fs_dwt
t_dwt = np.linspace(0, T_dwt, nsamples_dwt, endpoint=False)
a_dwt = 0.02
f0_dwt = 128
def signal_pro_dwt(data):
    mean_value = 0
    # do the bandpass filter
    for i in range(40):
        for j in range(32):
            data[i][j] = butter_bandpass_filter(data[i][j], lowcut_dwt, highcut_dwt, fs_dwt, order=5)
    # creating dummy variable which contains same data information 
    error_eye =  np.zeros((40,32,7680))
    new_data =  np.zeros((40,32,7680))
    for i in range(40):
        for j in range(32):
            for k in range(7680):
                error_eye[i][j][k] = data[i][j][k]
                new_data[i][j][k] = data[i][j][k]
    for i in range(40):
        error_eye[i] = eye_movement_artifact(error_eye[i])
    for i in range(40):
        for j in range(32):
            mean_value = np.mean(error_eye[i][j])
            for k in range(7680):
                if(data[i][j][k] > 0.0):
                    # data is positive
                    new_data[i][j][k] = data[i][j][k] - abs(mean_value)
                else: # data is negative
                    new_data[i][j][k] = data[i][j][k] + abs(mean_value)
    return new_data

In [25]:
def main_function(subject, channel_name):
  deap_dataset = pickle.load(open('/content/drive/MyDrive/Deap/' + subject + '.dat','rb'),encoding = 'latin1')
  labels, data = deap_dataset['labels'], deap_dataset['data']
  data = data[0:40 , 0:32 , 384:8064]
  filter_data = signal_pro(data)
  channel_no = get_channel_no(channel_name)
  data_dwt = signal_pro_dwt(data)
  features = get_features(data, data_dwt, channel_no)
  channels_name = get_channels_name(channel_no)
  df = pd.DataFrame(features, columns = channels_name)
  mypath = "/content/drive/MyDrive/Sequential methods for channel selection/our code/New/" + subject + "_arousal.csv"
  df.to_csv(mypath, index = False, encoding = 'utf-8-sig')

In [ ]:
counter = 0
for subject in subject_names[30:]:
  main_function(subject, subject_arousal_channels[counter])
  counter =  counter + 1
  print("Done: ", subject)